# Example usage

The Viam SDK can be used in two ways:
1. As a client to connect to a (remote or local) robot
2. To create custom components and provide additional functionality to a robot

## Connect as a client

To connect to a robot as a client, you should instantiate an instance of a `RobotClient`

In [1]:
# Please excuse the boilerplate
%autoawait asyncio
import warnings
warnings.filterwarnings('ignore')

In [2]:
from viam import logging
from viam.robot.client import RobotClient
from viam.rpc.dial import DialOptions

async def connect() -> RobotClient:
    options = RobotClient.Options(
        dial_options=DialOptions(insecure=True),
        log_level=logging.FATAL
    )
    return await RobotClient.at_address('0.0.0.0:5000', options)

robot = await connect()
robot.resource_names

StreamTerminatedError: Connection lost

You can also create a `RobotClient` by providing an existing connection

In [3]:
from viam import logging
from viam.robot.client import RobotClient
from viam.rpc.dial import DialOptions, dial_direct

async def connect_with_channel() -> RobotClient:
    async with await dial_direct('localhost:9090', DialOptions(insecure=True)) as channel:
        return await RobotClient.with_channel(channel, RobotClient.Options(refresh_interval=10, log_level=logging.FATAL))

Once you have a connected `RobotClient`, you can then obtain the robot's components by their name

In [9]:
from viam.components.motor import Motor

async def motor_control():
    robot = await connect()
    motor = Motor.from_robot(robot, 'motor0')
    await motor.go_to(rpm=100, position_revolutions=1)
    await robot.close()

Unclosed connection: Channel('localhost', 9090, ..., path=None)


You can also use the `RobotClient` to make service calls to the connected robot.

In [ ]:
from viam.services.types import ServiceType

async def vision():
    vision_service = robot.get_service(ServiceType.VISION)
    detectors = await vision_service.get_detections('camera_1', 'detector_1')

At the end, don't forget to close the connection

In [5]:
async def cleanup():
    await robot.close()

## Create custom components

While the main RDK is written in golang, you can create custom components in python and connect them to a robot as a `remote` component. This allows you to extend the functionality of a robot, or even create an entire robot exclusively in python.

The steps required in creating a custom component and connecting it to a robot are
1. Subclass a component and implement desired functions
2. Create an `rpc.server.Server` instance and register the custom component
3. Start the `Server` and register the running server as a remote

### Subclass a component

The SDK provides a wide array of components to customize. You can browse through the API Reference to see all of them, but for now we'll use an `Arm` as an example. Our custom Arm will be extremely simple -- it will only save the positions provided to it.

In [1]:
from typing import Optional
from viam.components.arm import Arm, JointPositions, Pose, WorldState

class MyCoolArm(Arm):

    def __init__(self, name: str):
        # Starting position
        self.position = Pose(
            x=0,
            y=0,
            z=0,
            o_x=0,
            o_y=0,
            o_z=0,
            theta=0,
        )

        # Starting joint positions
        self.joint_positions = JointPositions(degrees=[0, 0, 0, 0, 0, 0])
        self.is_stoppped = True
        super().__init__(name)

    async def get_end_position(self) -> Pose:
        return self.position

    async def move_to_position(
        self, pose: Pose,
        world_state: Optional[WorldState] = None
    ):
        self.is_stoppped = False
        self.position = pose

    async def get_joint_positions(self) -> JointPositions:
        return self.joint_positions

    async def move_to_joint_positions(self, positions: JointPositions):
        self.is_stoppped = False
        self.joint_positions = positions

    async def stop(self):
        self.is_stoppped = True

You can view more component implementations in the [examples](https://github.com/viamrobotics/python-sdk/blob/main/examples/server/v1/components.py).

### Register the custom component

Now that we've created the custom component, we must register it with a server so that it will be visible to any robots connecting to it.

In [12]:
# python-server.py
import asyncio
from viam.rpc.server import Server

async def main():
    srv = Server([MyCoolArm('my-arm')])

if __name__ == '__main__':
    try:
        asyncio.run(main())
    except:
        pass

### Start the Server and add it as a Remote

Now that we have a server that knows about our custom Arm component, we need to start the server so it can receive gRPC calls. Once it's started, we can then register this server as a remote.

```python3
# python-server.py

async def main():
  ...
  await srv.serve()
```
**NB**: When you call `srv.serve()`, the default host and port is `localhost:9090`. This can be changed by passing in a `host` and/or `port` parameter to the `serve` function.

To use this custom server as part of a larger robot, you’ll want to add it as a `remote` in the config for your main part.
```json
[
    {
      "name": "my-cool-python-components", // The name of the remote, can be anything
      "insecure": true,                    // Whether this connection should use SSL
      "address": "localhost:9090"          // The location of the remote robot
    }
  ]
```

And to ensure that the python server starts up with the rest of the robot, you can add it as a process. 
```json
[
  {
    "id": 0,
    "log": true,
    "name": "python",
    "args": [
      "/home/pi/python-server.py"
    ]
  }
]
```

**NB**: The `viam-server` starts as a root process, so you may need to switch users to run the python SDK server.
```json
[
  {
    "id": 0,
    "log": true,
    "name": "sudo",
    "args": [
      "-u",
      "pi",
      "python",
      "/home/pi/python-server.py"
    ]
  }
]
```
